In [1]:
import numpy as np
import sklearn.neural_network as nn
from sklearn.externals import joblib

In [2]:
net = nn.MLPRegressor(solver='lbfgs', alpha=1e-5,
                hidden_layer_sizes=(20, 40, 43, 40, 20, 20), random_state=1, max_iter=10000)
net

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 40, 43, 40, 20, 20),
       learning_rate='constant', learning_rate_init=0.001, max_iter=10000,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5, random_state=1,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [3]:
lista = [
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mBack1.txt', 60.0, 3.96, 2.7, 3.48],
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mBack2.txt', 60.0, 3.96, 2.7, 3.48],
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mBack3.txt', 59.0, 3.96, 2.7, 3.48],
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mBack4.txt', 62.0, 3.96, 2.7, 3.48],
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mBack5.txt', 56.0, 3.96, 2.7, 3.48],
['/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Vision/Dist/mFront1.txt', 60.0, 3.96, 2.7, 1.94],
]

data_in_all = [[0]]
data_out_all = [[0]]

for name, Tmax, po, p, dist_stop in lista:
    data = np.loadtxt(name, delimiter = ', ')

    robot_mov = data[data[:,1] != 0][:,1:]

    for i in xrange(len(robot_mov)):
        if robot_mov[i,1] < Tmax/2:
            robot_mov[i,1] = po + (2*robot_mov[i,1]*(p - po))/Tmax
        else:
            robot_mov[i,1] = p + (2*(robot_mov[i,1] - 30)*(po - p))/Tmax

    data_in = np.transpose([robot_mov[:,0]])
    data_out = np.transpose([robot_mov[:,1]])

    data_in = np.concatenate((data_in, np.transpose([data[data[:,0] != 0][:,0]])), axis=0)

    data_out = np.concatenate((data_out, np.transpose([np.ones(len(data[data[:,0] != 0][:,0]))*dist_stop])), axis=0)
    
    data_in_all = np.concatenate((data_in_all, data_in), axis=0)

    data_out_all = np.concatenate((data_out_all, data_out), axis=0)

net.fit(data_in_all[1:,:], data_out_all[1:,:])

print "Min:", min(abs(net.predict(data_in)-np.transpose(data_out)[0]))*100, "cm"
print "Media:", np.mean(abs(net.predict(data_in)-np.transpose(data_out)[0]))*100, "cm"
print "Max:", max(abs(net.predict(data_in)-np.transpose(data_out)[0]))*100, "cm"

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1266: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Min: 0.0273688004183 cm
Media: 1.99274120858 cm
Max: 28.7372284755 cm


In [4]:
# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(net, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.predict(0.17)
print(result)

[ 3.45512607]


In [5]:
len(data_in_all)

791